In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

# Open browser

In [34]:
# 크롬드라이버 실행. 4.6 버전부터 크롬드라이버 별도 다운로드 및 경로 설정 불필요
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
site = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
driver.get(site)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# Get community review list

In [35]:
# Open community review window by clicking Show more reviews
path = "//button[@tabindex='0']"
#path = "//span[text()='Show more reviews']" # successful
#path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Show reviews by recent
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Get reviews

In [94]:
class_review = "communityReview__reviewText--2bfLj"
class_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"
class_date = class_date.replace(' ', '.')
   

max_rev = 100
max_scr = 5
time_scr = 2

def extract_text(class_name, driver=driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


reviews = list()
dates = list()
n_scr = 0

while True:
    list_r = extract_text(class_review)
    list_d = extract_text(class_date)

    n = len(list_r) - len(reviews)
    if n > 0: # get new n reviews
        reviews.extend(list_r[-n:])
        dates.extend(list_d[-n:])
        n_scr = 0
    else:
        n_scr += 1

    if ((len(reviews) > max_rev) or (n_scr > max_scr)):
        break
    else:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(time_scr)

#len(reviews), len(dates)

# close browser
driver.close()

# Save reviews

In [95]:
import pandas as pd

df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
df_reviews['date'] = pd.to_datetime(df_reviews['date'])
df_reviews.head()

,date,review
0,2024-01-24,Little too cherry on the front end for me
1,2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m..."
2,2024-01-19,Aight
3,2024-01-18,무난한 맛 가성비 좋은듯
4,2024-01-17,oak cherry black cherry chocolate blackcurrant...


In [84]:
f = 'wine_reviews'
df_reviews.to_csv(f, index=False)  

# Module

In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.common.exceptions import ElementNotVisibleException, StaleElementReferenceException, NoSuchElementException

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import pandas as pd 
import numpy as np
from tqdm import tqdm
from datetime import datetime

In [32]:
def collect_reviews(class_name, driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]
    

def vivino_reviews(wine_url, wine_name, 
                   end_date = '20230101',
                   max_rev = 1e4, 
                   max_scr = 10, # max num of scroll for new review list 
                   time_scr = 2, # time to wait after scrolling. StaleElementReferenceException if not enough
                   loc1=None, 
                   loc2=None, 
                   loc3=None, 
                   loc4=None,
                   # take the final review list as the review page has same old reiviews at the end every update by scroll-down 
                   final_only = True,
                   check_idx = 4, # community review list has 3 old reviews at the end of list every loading of new reviews
                   rev_date_format = '%b %d, %Y',
                   #headless = False # fail to locate loc1 if set to True
                  ):

    #if headless:
    if False:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
    else:
        options = None

    driver = webdriver.Chrome(options=options) 
    driver.get(wine_url)
    time.sleep(3)

    loc3 = loc3.replace(' ', '.')
    loc4 = loc4.replace(' ', '.')

    try:
        # enter community reviews
        l = loc1
        driver.find_element(By.XPATH, loc1).click()
        time.sleep(1)
        
        # Show reviews by recent
        l = loc2
        driver.find_element(By.XPATH, loc2).click()
        time.sleep(1)

        l = loc3
        list_r = collect_reviews(loc3, driver)
        l = loc4
        list_d = collect_reviews(loc4, driver)
    except:
        print(f'ERROR) Check locator: {l}')
        driver.quit()
        return None
     
    reviews = list()
    dates = list()
    n_scr = 0

    #pbar = tqdm(total=max_rev, position=0)
    pbar = tqdm(position=0)
    
    while True:
        n = len(list_r) - len(reviews)
        if n > 0: # get new n reviews
            if final_only: # replace review list with the latest scan
                reviews = list_r
                dates = list_d
            else: # update review list with new reviews from the latest scan
                reviews.extend(list_r[-n:])
                dates.extend(list_d[-n:])
            n_scr = 0
            pbar.update(n)
        else:
            n_scr += 1

        # testing
        #return list_d

        d1 = datetime.strptime(list_d[-check_idx], rev_date_format)
        d2 = datetime.strptime(end_date, '%Y%m%d')
        #print('testing:', d1)

        if ((len(reviews) > max_rev) or (d1 < d2)):
            print(f'{len(reviews)} reviews collected.')
            break
        elif (n_scr > max_scr):
            # redundunt as n_try checking max_scr as well?
            print(f'WARNING: No additional reviews after {max_scr} reloading.')
            break
        else:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(time_scr)

            failed = True
            n_try = 0
            while failed:
                try:
                    list_r = collect_reviews(loc3, driver)
                    list_d = collect_reviews(loc4, driver)
                    failed = False
                except StaleElementReferenceException:
                    time.sleep(time_scr)
                    n_try += 1
                    
                if n_try > max_scr:
                    failed = False
                    print('WARNING: fail to collect all reviews')

    pbar.close()

    # close browser
    driver.quit()

    # save result
    df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])
    
    if False: #deprecated
        df_reviews.to_csv(filename, index=False)  

    return df_reviews


def generate_wine_id(df_reviews):
    if df_reviews.empty:
        wid = 0
    else:
        wid = df_reviews.id.max()
        wid = 0 if wid is np.nan else wid+1 # check if no data row
    return wid


def concat_reviews(df_reviews, df, wine_name, col_rev, save=True):
    wine_id = generate_wine_id(df_reviews)
    df[['id', 'wine']] = [wine_id, wine_name]
    df = df.reindex(columns=col_rev)
    if save:
        df.to_csv(f'wine_{wine_id}.csv', index=False)
    
    df_reviews = pd.concat([df_reviews if not df_reviews.empty else None, df])
    return df_reviews

In [33]:
locator_more = "//button[@tabindex='0']"
locator_recent = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

locator_review = "communityReview__reviewText--2bfLj"
locator_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"

kwargs = {
    'loc1': locator_more,
    'loc2': locator_recent, 
    'loc3': locator_review,
    'loc4': locator_date
}

In [34]:
kwargs.update({
    'end_date': '20220101',
    #'max_scr': 10, 'time_scr': 2, 'check_idx': 4,
})

In [ ]:
wines = list()
wines.append([
    'Casillero del Diablo Cabernet Sauvignon (Reserva)',
    'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
])
wines.append([
    'Yellow Tail Cabernet Sauvignon',
    'https://www.vivino.com/yellow-tail-cabernet-sauvignon/w/2549?ref=nav-search#all_reviews'
])

In [ ]:
df_reviews = pd.DataFrame()
col_rev = ['id', 'wine', 'date', 'review']

for wine_name, wine_url in wines:
    df = vivino_reviews(wine_url, wine_name, **kwargs)
    df_reviews = concat_reviews(df_reviews, df, wine_name, col_rev, save=True)

In [216]:
!zip wine_reviews.zip wine*csv

  adding: wine_0.csv (deflated 67%)
  adding: wine_1.csv (deflated 57%)


# lab